In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np

In [214]:
seqv=pd.read_table("/groups/nordborg/user/elizaveta.grigoreva/genome_evolution_pacbio/analyses/004_polymorphism_annas_snps/hmp_parsed/pi_by_hand/chr1_27acc.txt")
#with open("/groups/nordborg/user/elizaveta.grigoreva/genome_evolution_pacbio/analyses/004_polymorphism_annas_snps/hmp_parsed/pi_by_hand/chr1_27acc.txt") as txt:
#     text = txt.readlines()
type(seqv)

pandas.core.frame.DataFrame

In [215]:
position=seqv['POS']

In [182]:
#Transpose matrix
test_transpose = np.transpose(seqv)

In [147]:
type(test_transpose)

pandas.core.frame.DataFrame

In [324]:
test_transpose=test_transpose.drop('POS')

In [151]:
#Calculate number of differences in each row 
differences_list=[]
comparisons=0
#Iterate over each column 
for f in range(0, len(test_transpose.columns)):
    differences=0
    for i in range(len(test_transpose[f])):
        if str(test_transpose[f][i]) != 'nan':
            for j in range(i,len(test_transpose[0])):
                if str(test_transpose[f][j]) != 'nan':
                    if test_transpose[f][j]!= test_transpose[f][i]:
                        differences+= 1
    differences_list.append(differences)

In [326]:
###Number of comparisons-number of letters
comparisons=test_transpose.count()

In [327]:
comparisons.head()

0    4
1    4
2    5
3    5
4    5
dtype: int64

In [328]:
#Calculate combinations (n over k)
from math import comb
combinations=[]
for i in comparisons:
    combine=comb(i,2)
    combinations.append(combine)

In [285]:
len(chr1_length)

30427671

In [340]:
#Calculate Pi nominator and denominator in sliding windows 
import numpy as np
import math
window_size = 200000
i = 0
#Move combinations to type int 
numbers = [0 if math.isnan(x) else x for x in list_combinations]
list_pi = [0 if math.isnan(x) else x for x in list_pi]
numbers = [ int(x) for x in numbers ]
# Initialize an empty list to store moving averages
moving_averages_num = [] #initialize numerator 
moving_averages_denum = [] #initialize denumirator
a=[]
b=[]
# Loop through the array t o
#consider every window of size 
while i < len(chr1_length) - window_size + 50000:
    a.append(i)
    b.append(i+50000)
    # Calculate the average of current window
    window_average_num = round(np.sum(list_pi[
      i:i+window_size]) / window_size, 2)
    window_averages_denum = round(np.sum(numbers[
      i:i+window_size]) / window_size, 2)
    # Store the average of current
    # window in moving average list
    moving_averages_num.append(window_average_num)
    moving_averages_denum.append(window_averages_denum)
    # Shift window to right by one position
    i += 50000


In [341]:
#Calculate PI in sliding windows
pi_raw = [x / y for x, y in zip(moving_averages_num, moving_averages_denum)]
##Add column of coordinates (bins)
pi_wind=[i/100000 for i in (pi_raw)]

In [342]:
#Zip everything in dataframe
zipped = list(zip(a, b, pi_wind))#Make one dataframe
df = pd.DataFrame(zipped, columns=['start', 'stop', 'pi'])
df

,start,stop,pi
0,0,50000,0.000002
1,50000,100000,0.000002
2,100000,150000,0.000002
3,150000,200000,0.000002
4,200000,250000,0.000002
...,...,...,...
601,30050000,30100000,0.000002
602,30100000,30150000,0.000002
603,30150000,30200000,0.000002
604,30200000,30250000,0.000002


In [343]:
#Write to dataframe
df.to_csv (r'/groups/nordborg/user/elizaveta.grigoreva/genome_evolution_pacbio/analyses/004_polymorphism_annas_snps/hmp_parsed/pi_by_hand/pi_chr1.csv', index = None, header=True) 

In [241]:
##Create chromosome sequence length 
import numpy as np
chr1_length=np.arange(1,30427671+1)
chr1_length = pd.DataFrame(chr1_length, columns = ['POS'])
chr1_length['PI'] = 0 #Add PI column

,POS,PI
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
30427666,30427667,0
30427667,30427668,0
30427668,30427669,0
30427669,30427670,0


In [332]:
#Merge with difference and combinations list 
difference=pd.DataFrame(differences_list, columns = ['PI'])
difference['POS']=pd.DataFrame(position, columns = ['POS']).astype(int)
difference['comb']=pd.DataFrame(combinations, columns = ['comb']).astype(int)

In [333]:
#Merge positions and update PI
inner_merged_total = pd.merge(chr1_length, difference, how="left", on=["POS"])

In [334]:
inner_merged_total.head(10)
#Extract column as a list
list_pi=inner_merged_total['PI_y']
list_combinations=inner_merged_total['comb']

In [335]:
list_pi=list_pi.tolist()
list_combinations=list_combinations.tolist()

In [ ]:
###Checking genome letters
with open("/groups/nordborg/user/elizaveta.grigoreva/genome_evolution_pacbio/data/tair10/GCF_000001735.4_TAIR10.1_genomic_5Chr_renamd_for_syri.fna") as raw:
    raw_lines = raw.readlines()
#Create reference dictinoary  
ref_chromosome_dict={}
for line in raw_lines:
    line=line.strip()
    if line.startswith(">"):
        chromosome=line[1:] #First line
        ref_chromosome_dict[chromosome]=['_']
        continue 
    ref_chromosome_dict[chromosome].append(line.upper())
for chromosome in ref_chromosome_dict:
    ref_chromosome_dict[chromosome] = "".join(ref_chromosome_dict[chromosome])
print(ref_chromosome_dict['Chr1'][23259])
